In [390]:
import pandas as pd
import pypyodbc as podbc
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
from scipy.sparse import csr_matrix
from collections import OrderedDict

In [371]:
#pandas params
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

In [3]:
#Connect to RDS db
conn = podbc.connect(
    #Trusted_Connection='Yes',
    Driver='{SQL Server}',
    Server='nbahistorical.cmgzaupuq9dz.us-east-2.rds.amazonaws.com',
    UID='admin',
    PWD='Gs239350',
    Database='pbp',
    MultipleActiveResultSets = 'True'
)

In [4]:
df = pd.read_sql_query(sql = """
SELECT
	* FROM (
SELECT 
	EVENTNUM AS E, 
	HOMEDESCRIPTION,
	VISITORDESCRIPTION,
	EVENTMSGTYPE,
	GAME_ID,
	STRING_AGG(PERSON_ID, ', ') 
		WITHIN GROUP (ORDER BY TEAM_ID,PERSON_ID)
	AS STINT,
	ROW_NUMBER () 
		OVER (PARTITION BY GAME_ID, EVENTNUM ORDER BY GAME_ID) AS R 
FROM
		(
		SELECT  
			*,
			(CASE
			--Method for time during regulation except at beginning of quarter, with >= 10 minutes on the clock
			WHEN PERIOD <5 AND PCTIMESTRING NOT LIKE '12:0%' AND PCTIMESTRING LIKE '__:%'
			THEN
			(((CAST(period AS int) - 1) * 7200)
					+ 
					(((CAST(SUBSTRING(PCTIMESTRING,1,2) AS int)
						-11) 
							*(-1))
								*600)
					+
					((CAST(SUBSTRING(PCTIMESTRING,4,2) AS int)
						-60)
							*(-10)))
			--Method for time during regulation except at beginning of quarter, with < 10 minutes on the clock
			WHEN PERIOD <5 AND PCTIMESTRING NOT LIKE '12:0%' AND PCTIMESTRING NOT LIKE '__:%'
			THEN
			(((CAST(period AS int) - 1) * 7200)
					+ 
					(((CAST(SUBSTRING(PCTIMESTRING,1,1) AS int)
						-11) 
							*(-1))
								*600)
					+
					((CAST(SUBSTRING(PCTIMESTRING,4,2) AS int)
						-60)
							*(-10)))
		--Method for time during regulation at beginning of quarter only
			WHEN PERIOD <5 AND PCTIMESTRING LIKE '12:0%'
			THEN
			(
			(
				((CAST(period AS int)) - 1)
				* 7200)
					+ (
					((((CAST(SUBSTRING(PCTIMESTRING,1,2) AS int))
						-12)
							*(-1))
								*60)
				)
			)	
		--Method for time during overtime 
			WHEN PERIOD >4
			THEN
			(28800 +
			(((CAST(period AS int)) - 5) * 3000)
					+ 
					((((CAST(SUBSTRING(PCTIMESTRING,1,1) AS int))
						-4)
							*(-1))
								*600)
					+
					(((CAST(SUBSTRING(PCTIMESTRING,4,2) AS int))
						-60)
							*(-10))
				)
			ELSE 'ERROR'
			END)
			AS TIME
		FROM 	
			all_pbp
		)
		play_by_play
			LEFT JOIN
				(
				SELECT
					GAME_ID AS GAME,
					CAST(TEAM_ID AS int) AS TEAM_ID,
					TEAM_CITY,
					TEAM_NAME,
					PERSON_ID,
					PLAYER_NAME,
					CAST(TIME_IN AS int) AS TIME_IN,
					CAST(TIME_OUT AS int) AS TIME_OUT,
					PLAYER_POINTS,
					POINT_MARGIN,
					USAGE
				FROM
					rotations
				) b
					ON play_by_play.GAME_ID = b.GAME
					AND TIME < TIME_OUT
					AND TIME >= TIME_IN
	GROUP BY 
		GAME_ID,
		PERIOD,
		EVENTNUM,
		HOMEDESCRIPTION,
		VISITORDESCRIPTION,
		EVENTMSGTYPE
) q
WHERE 
	GAME_ID LIKE '214%'
	AND R = 1
;
""", con = conn)

In [224]:
pbp = pd.read_sql_query(sql = """
SELECT 
   GAME_ID, EVENTNUM, EVENTMSGTYPE, PERIOD, PCTIMESTRING, HOMEDESCRIPTION, VISITORDESCRIPTION, SCORE, SCOREMARGIN, 
   PLAYER1_ID, PLAYER1_TEAM_ID
FROM 
    all_pbp
WHERE
    GAME_ID LIKE '214%';
""", con = conn)

In [225]:
pbp.columns = map(str.upper, pbp.columns)

#Converting EVENTNUM to int, then creating true event number to cover gaps
pbp['EVENTNUM'] = pbp['EVENTNUM'].astype(int)
pbp['EVENT'] = pbp.groupby(['GAME_ID'])['EVENTNUM'].rank()

#Manipulating SCOREMARGIN
pbp['SCOREMARGIN'] = pbp['SCOREMARGIN'].replace('',np.nan)
pbp['SCOREMARGIN'] = pbp.groupby(['GAME_ID'])['SCOREMARGIN'].fillna(method = 'ffill')
pbp['SCOREMARGIN'] = np.where(pbp['SCOREMARGIN'] == 'TIE', 0, pbp['SCOREMARGIN'])
pbp['SCOREMARGIN'] = pbp['SCOREMARGIN'].fillna(0)

#Manipulating SCORE
pbp['SCORE'] = pbp['SCORE'].replace('',np.nan)
pbp['SCORE'] = pbp.groupby(['GAME_ID'])['SCORE'].fillna(method = 'ffill')
pbp['SCORE'] = pbp['SCORE'].fillna('0 - 0')

#Create HOME_SCORE and AWAY_SCORE
pbp.insert(loc = 11, column = 'HOME_SCORE', value = [i[1] for i in pbp['SCORE'].str.split(' - ')])
pbp.insert(loc = 12, column = 'AWAY_SCORE', value = [i[0] for i in pbp['SCORE'].str.split(' - ')])

#Create NEXT_EVENTNUM, join associated data for next event to current event  
pbp.insert(loc = 13, column = 'NEXT_EVENTNUM', value = pbp['EVENT'].astype(int) + 1)
event = pbp[['GAME_ID','EVENT','HOMEDESCRIPTION','VISITORDESCRIPTION']]
#event['EVENTNUM'] = event['EVENTNUM'].astype(int)
event = event.rename(columns = {"EVENT":"NEXT_EVENTNUM",#"EVENT":"NEXT_EVENT",
                                "HOMEDESCRIPTION":"NEXT_HOMEDESCRIPTION","VISITORDESCRIPTION":"NEXT_VISITORDESCRIPTION"})
pbp = pbp.merge(right = event, how = 'left', on = ['GAME_ID','NEXT_EVENTNUM'])

#Repeat the process for the preceding event
pbp.insert(loc = 14, column = 'LAST_EVENTNUM', value = pbp['EVENT'].astype(int) - 1)
event = event.rename(columns = {"NEXT_EVENTNUM":"LAST_EVENTNUM",#"NEXT_EVENT":"LAST_EVENT",
                               "NEXT_HOMEDESCRIPTION":"LAST_HOMEDESCRIPTION","NEXT_VISITORDESCRIPTION":"LAST_VISITORDESCRIPTION"})
#pbp = pbp.merge(right = event, how = 'left', on = [['GAME_ID','LAST_EVENTNUM']])
pbp = pbp.merge(right = event, how = 'left', left_on = ['GAME_ID','LAST_EVENTNUM'], right_on = ['GAME_ID','LAST_EVENTNUM'])

C:\Users\gsteele\Miniconda3\lib\site-packages\pandas\core\reshape\merge.py:1123: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [235]:
#Transform time to be consistent with rotations data
pbp.insert(loc = 16, column = 'MINUTES', value = [i[0] for i in pbp['PCTIMESTRING'].str.split(':')])
pbp.insert(loc = 16, column = 'SECONDS', value = [i[1] for i in pbp['PCTIMESTRING'].str.split(':')])

#Transform time suring regulation except at beginning of period
pbp['TIME'] = np.where(((pbp['PERIOD'].astype(int) < 5) & (pbp['PCTIMESTRING'] != '12:00')), 
                      (((pbp['PERIOD'].astype(int) -1) *7200) 
                      + ((pbp['MINUTES'].astype(int) -11) * (-600))
                      + ((pbp['SECONDS'].astype(int) -60) * (-10))),
                          pbp['PCTIMESTRING'])
#Transform time during regulation at beginning of period
pbp['TIME'] = np.where(((pbp['PERIOD'].astype(int) < 5) & (pbp['PCTIMESTRING'] == '12:00')), 
                      (((pbp['PERIOD'].astype(int) -1) *7200) 
                      + ((pbp['MINUTES'].astype(int) -12) * (-600))),
                          pbp['TIME'])
#Transform time during overtime
pbp['TIME'] = np.where(pbp['PERIOD'].astype(int) > 4, 
                      (28800 + 
                      ((pbp['PERIOD'].astype(int) -5) *3000) 
                      + ((pbp['MINUTES'].astype(int) -4) * (-600))
                      + ((pbp['SECONDS'].astype(int) -60) * (-10))),
                          pbp['TIME'])

In [417]:
len(pbp[(pbp['NEXT_HOMEDESCRIPTION'].isnull() == True)
    & (pbp['TIME'] != 28800) & (pbp['PCTIMESTRING'] != '0:00')])
#pbp.sort_values(by = ['GAME_ID','EVENT']).head()

413

In [247]:
df['e'] = df['e'].astype(int)
pbp_df = df.merge(right = pbp, how = 'left', left_on = ['game_id','e'], right_on = ['GAME_ID','EVENTNUM'])

In [248]:
#len(pbp_df[pbp_df['stint'].isna() == True])
pbp_df['DESCRIPTION'] = (pbp_df['HOMEDESCRIPTION'] + pbp_df['VISITORDESCRIPTION'])
pbp_df['NEXT_DESCRIPTION'] = (pbp_df['NEXT_HOMEDESCRIPTION'] + pbp_df['NEXT_VISITORDESCRIPTION'])
pbp_df['LAST_DESCRIPTION'] = (pbp_df['LAST_HOMEDESCRIPTION'] + pbp_df['LAST_VISITORDESCRIPTION'])

#Away ball
pbp_df['HA'] = np.where((pbp_df['HOMEDESCRIPTION'] == '') 
                        & (pbp_df['EVENTMSGTYPE'] != '2') 
                        & (pbp_df['EVENTMSGTYPE'] != '5'), 'A', 0)
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '5') & (pbp_df['HOMEDESCRIPTION'].str.contains('STEAL')), 
                       'A', pbp_df['HA'])
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') & (pbp_df['HOMEDESCRIPTION'].str.contains('BLOCK')), 
                       'A', pbp_df['HA'])  
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') & (pbp_df['HOMEDESCRIPTION'] == ''), 
                       'A', pbp_df['HA']) 
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '5') & (pbp_df['HOMEDESCRIPTION'] == ''), 
                       'A', pbp_df['HA']) 
pbp_df['HA'] = np.where(pbp_df['HOMEDESCRIPTION'].str.contains('S.FOUL'), 'A', pbp_df['HA'])

#Home ball
pbp_df['HA'] = np.where((pbp_df['VISITORDESCRIPTION'] == '') & (pbp_df['EVENTMSGTYPE'] != '2') & (pbp_df['EVENTMSGTYPE'] != '5')
                                                                    , 'H', pbp_df['HA'])
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '5') & (pbp_df['VISITORDESCRIPTION'].str.contains('STEAL')), 
                       'A', pbp_df['HA'])
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') & (pbp_df['VISITORDESCRIPTION'].str.contains('BLOCK')), 
                       'A', pbp_df['HA'])  
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') & (pbp_df['VISITORDESCRIPTION'] == ''), 
                       'A', pbp_df['HA'])  
pbp_df['HA'] = np.where((pbp_df['EVENTMSGTYPE'] == '5') & (pbp_df['VISITORDESCRIPTION'] == ''), 
                       'A', pbp_df['HA'])  
pbp_df['HA'] = np.where(pbp_df['VISITORDESCRIPTION'].str.contains('S.FOUL'), 'H', pbp_df['HA'])

###Counting Possessions
#Turnovers and violations
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '5') | (pbp_df['EVENTMSGTYPE'] == '7'), 1, 0)
#Made basket without a shooting foul
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '1') 
    & (pbp_df['HOMEDESCRIPTION'] == '')
   & (pbp_df['NEXT_HOMEDESCRIPTION'].str.contains('S.FOUL') == False), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '1') 
    & (pbp_df['VISITORDESCRIPTION'] == '')
   & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('S.FOUL') == False), 1, pbp_df['POSS'])
#Made final free throw
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') 
                          & (pbp_df['DESCRIPTION'].str.contains('1 of 1'))
                          & (pbp_df['DESCRIPTION'].str.contains('MISS') == False), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') 
                          & (pbp_df['DESCRIPTION'].str.contains('2 of 2'))
                          & (pbp_df['DESCRIPTION'].str.contains('MISS') == False), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') 
                          & (pbp_df['DESCRIPTION'].str.contains('3 of 3'))
                          & (pbp_df['DESCRIPTION'].str.contains('MISS') == False), 1, pbp_df['POSS'])
#Defensive rebound
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['HOMEDESCRIPTION'] == '')
                          & (pbp_df['NEXT_HOMEDESCRIPTION'].str.contains('REBOUND')), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['VISITORDESCRIPTION'] == '')
                          & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('REBOUND')), 1, pbp_df['POSS'])
#Team defensive rebound
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['HOMEDESCRIPTION'] == '')
                          & (pbp_df['NEXT_HOMEDESCRIPTION'].str.contains('Rebound')), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['VISITORDESCRIPTION'] == '')
                          & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('Rebound')), 1, pbp_df['POSS'])
#Block defensive rebounds
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['HOMEDESCRIPTION'].str.contains('BLOCK'))
                          & (pbp_df['NEXT_HOMEDESCRIPTION'].str.contains('REBOUND')), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '2') 
                          & (pbp_df['VISITORDESCRIPTION'].str.contains('BLOCK'))
                          & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('REBOUND')), 1, pbp_df['POSS'])
#Free throw rebounds
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') 
                          & (pbp_df['DESCRIPTION'].str.contains('MISS'))
                          & (pbp_df['NEXT_HOMEDESCRIPTION'].str.contains('REBOUND'))
                          & (pbp_df['HOMEDESCRIPTION'] == ''), 1, pbp_df['POSS'])
pbp_df['POSS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') 
                          & (pbp_df['DESCRIPTION'].str.contains('MISS'))
                          & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('REBOUND'))
                          & (pbp_df['VISITORDESCRIPTION'] == ''), 1, pbp_df['POSS'])
pbp_df['STINT_POSS'] = pbp_df.groupby(['GAME_ID','PERIOD','stint','HA'])['POSS'].transform('sum')

###Calculating Points
pbp_df['PTS'] = np.where((pbp_df['EVENTMSGTYPE'] == '3') & (pbp_df['DESCRIPTION'].str.contains('MISS') == False), 1, 0)
pbp_df['PTS'] = np.where((pbp_df['EVENTMSGTYPE'] == '1') 
                         & (pbp_df['DESCRIPTION'].str.contains('3PT Jump')), 3, pbp_df['PTS'])
pbp_df['PTS'] = np.where((pbp_df['EVENTMSGTYPE'] == '1') 
                         & (pbp_df['DESCRIPTION'].str.contains('3PT Jump') == False), 2, pbp_df['PTS'])
pbp_df['STINT_PTS'] = pbp_df.groupby(['GAME_ID','PERIOD','stint','HA'])['PTS'].transform('sum')

#Team ids for home and away
pbp_df['HOME_TEAM'] = np.where((pbp_df['HOMEDESCRIPTION'] != '')
                               & (pbp_df['VISITORDESCRIPTION'] == '')
                               & (pbp_df['HOMEDESCRIPTION'] != 0)
                               & (pbp_df['HOMEDESCRIPTION'].isna() == False)
                               & (pbp_df['HOMEDESCRIPTION'].isnull() == False), pbp_df['PLAYER1_TEAM_ID'], 0)
pbp_df['AWAY_TEAM'] = np.where((pbp_df['VISITORDESCRIPTION'] != '')
                               & (pbp_df['HOMEDESCRIPTION'] == '')
                               & (pbp_df['VISITORDESCRIPTION'] != 0)
                               & (pbp_df['VISITORDESCRIPTION'].isna() == False)
                               & (pbp_df['VISITORDESCRIPTION'].isnull() == False), pbp_df['PLAYER1_TEAM_ID'], 0)

In [249]:
home_team = pbp_df[['game_id','HOME_TEAM']].drop_duplicates()
home_team = home_team[home_team['HOME_TEAM'] != 0]
home_team = home_team[home_team['HOME_TEAM'] != '']
away_team = pbp_df[['game_id','AWAY_TEAM']].drop_duplicates()
away_team = away_team[away_team['AWAY_TEAM'] != 0]
away_team = away_team[away_team['AWAY_TEAM'] != '']
team_dimension_table = home_team.merge(right = away_team, how = 'inner', on = ['game_id'])

In [298]:
stints = pbp_df[['game_id','stint','PERIOD','HA','STINT_POSS','STINT_PTS']].drop_duplicates()
stints = stints[stints['stint'].isna() == False]
stints = stints.merge(right = team_dimension_table, how = 'left', on = ['game_id'])
stints['ORATING'] = stints['STINT_PTS'] / stints['STINT_POSS'] * 100
stints['HOME_TEAM'] = stints['HOME_TEAM'].astype(float).astype(int)
stints['AWAY_TEAM'] = stints['AWAY_TEAM'].astype(float).astype(int)

In [407]:
lineups = []
home_ball_lineup = []
away_ball_lineup = []

#If the team with the lower TEAM_ID number has the ball
for i in stints[(stints['HOME_TEAM'] < stints['AWAY_TEAM']) & (stints['HA'] == 'H') & (stints['STINT_POSS'] >= 1)]['stint']:
    home_offense = OrderedDict({f"{j},offense": 1 for j in i.split(', ')[0:5]})
    away_defense = OrderedDict({f"{j},defense": (-1) for j in i.split(', ')[5:]})
    home_ball_lineup = home_offense.copy()
    home_ball_lineup.update(away_defense)
    lineups.append(home_ball_lineup)

for i in stints[(stints['AWAY_TEAM'] < stints['HOME_TEAM']) & (stints['HA'] == 'A') & (stints['STINT_POSS'] >= 1)]['stint']:
    away_offense = OrderedDict({f"{j},offense": 1 for j in i.split(', ')[0:5]})
    home_defense = OrderedDict({f"{j},defense": (-1) for j in i.split(', ')[5:]})
    away_ball_lineup = away_offense.copy()
    away_ball_lineup.update(home_defense)
    lineups.append(away_ball_lineup)

#If the team with the higher TEAM_ID number has the ball
for i in stints[(stints['HOME_TEAM'] > stints['AWAY_TEAM']) & (stints['HA'] == 'H') & (stints['STINT_POSS'] >= 1)]['stint']:
    home_offense_b = OrderedDict({f"{k},offense": 1 for k in [s for s in i.split(', ')[::-1]][0:5]})
    away_defense_b = OrderedDict({f"{k},defense": (-1) for k in [s for s in i.split(', ')[::-1]][5:]})
    home_ball_lineup_b = home_offense_b.copy()
    home_ball_lineup_b.update(away_defense_b)
    lineups.append(home_ball_lineup_b)

for i in stints[(stints['AWAY_TEAM'] > stints['HOME_TEAM']) & (stints['HA'] == 'A') & (stints['STINT_POSS'] >= 1)]['stint']:
    away_offense_b = OrderedDict({f"{k},offense": 1 for k in [s for s in i.split(', ')[::-1]][0:5]})
    home_defense_b = OrderedDict({f"{k},defense": (-1) for k in [s for s in i.split(', ')[::-1]][5:]})
    away_ball_lineup_b = away_offense_b.copy()
    away_ball_lineup_b.update(home_defense_b)
    lineups.append(away_ball_lineup)

orating = []
for s in stints[stints['STINT_POSS'] >= 1]['ORATING']:
    orating.append(s)

possessions = []
for s in stints[stints['STINT_POSS'] >= 1]['STINT_POSS']:
    possessions.append(s)


print(lineups[0:10])
#print(len(lineups))
#print(len(stints['stint']))
#print(len(orating))
#print(len(possessions))
#print(orating[0:25])
#print(possessions[0:25])

[OrderedDict([('201569,offense', 1), ('201600,offense', 1), ('201936,offense', 1), ('201950,offense', 1), ('203076,offense', 1), ('202696,defense', -1), ('202699,defense', -1), ('203095,defense', -1), ('203124,defense', -1), ('203901,defense', -1)]), OrderedDict([('201583,offense', 1), ('201600,offense', 1), ('202690,offense', 1), ('203085,offense', 1), ('2422,offense', 1), ('203473,defense', -1), ('203932,defense', -1), ('2557,defense', -1), ('2584,defense', -1), ('2732,defense', -1)]), OrderedDict([('201583,offense', 1), ('201600,offense', 1), ('201936,offense', 1), ('202690,offense', 1), ('203085,offense', 1), ('203473,defense', -1), ('203932,defense', -1), ('2557,defense', -1), ('2584,defense', -1), ('2732,defense', -1)]), OrderedDict([('201583,offense', 1), ('201600,offense', 1), ('201936,offense', 1), ('202690,offense', 1), ('203085,offense', 1), ('202696,defense', -1), ('203932,defense', -1), ('2557,defense', -1), ('2584,defense', -1), ('2732,defense', -1)]), OrderedDict([('2015

In [416]:
y = csr_matrix(orating)
y = y.transpose()
print(X.shape[0])
print(y.shape[0])

np.set_printoptions(threshold=np.inf)
v = DictVectorizer(sparse = True)    
X = v.fit_transform(lineups)

v.inverse_transform(X[0:25])

59948
59948


[{'201569,offense': 1.0,
  '201600,offense': 1.0,
  '201936,offense': 1.0,
  '201950,offense': 1.0,
  '202696,defense': -1.0,
  '202699,defense': -1.0,
  '203076,offense': 1.0,
  '203095,defense': -1.0,
  '203124,defense': -1.0,
  '203901,defense': -1.0},
 {'201583,offense': 1.0,
  '201600,offense': 1.0,
  '202690,offense': 1.0,
  '203085,offense': 1.0,
  '203473,defense': -1.0,
  '203932,defense': -1.0,
  '2422,offense': 1.0,
  '2557,defense': -1.0,
  '2584,defense': -1.0,
  '2732,defense': -1.0},
 {'201583,offense': 1.0,
  '201600,offense': 1.0,
  '201936,offense': 1.0,
  '202690,offense': 1.0,
  '203085,offense': 1.0,
  '203473,defense': -1.0,
  '203932,defense': -1.0,
  '2557,defense': -1.0,
  '2584,defense': -1.0,
  '2732,defense': -1.0},
 {'201583,offense': 1.0,
  '201600,offense': 1.0,
  '201936,offense': 1.0,
  '202690,offense': 1.0,
  '202696,defense': -1.0,
  '203085,offense': 1.0,
  '203932,defense': -1.0,
  '2557,defense': -1.0,
  '2584,defense': -1.0,
  '2732,defense': -1.

In [409]:
v = DictVectorizer()    
X = v.fit_transform(lineups)


lambdas = [.01, .025, .05, .1]
samples = X.shape[0]
alphas = [(l * samples / 2) for l in lambdas]

#clf = linear_model.RidgeCV(alphas = (np.array([0.01, 0.1, 1.0, 10, 100, 500, 750])), cv = 5)
clf = linear_model.RidgeCV(alphas = alphas, cv = 5)
model = clf.fit(X, orating, sample_weight = possessions)
#model = clf.fit(X, stints['ORATING'], sample_weight = stints['STINT_POSS'])

In [410]:
print(clf.alpha_)

2997.4


In [17]:
roster = pd.read_sql_query(sql = """
SELECT 
   DISTINCT PERSON_ID, PLAYER_NAME
FROM 
    rotations;
""", con = conn)

In [411]:
unfiltered_RAPM = []
player_ids = v.get_feature_names()
for pid in player_ids:
    unfiltered_RAPM.append((pid, model.coef_[player_ids.index(pid)]))

unfiltered_RAPM = pd.DataFrame.from_records(unfiltered_RAPM, columns = ['id','RAPM'])
unfiltered_RAPM.insert(loc = 1, column = 'ID', value = ([i[0] for i in unfiltered_RAPM['id'].str.split(',')]))
unfiltered_RAPM.insert(loc = 2, column = 'side', value = ([i[1] for i in unfiltered_RAPM['id'].str.split(',')]))

RAPM = unfiltered_RAPM.merge(right = roster, how = 'left', left_on = 'ID', right_on = 'person_id')

#O-RAPM / D-RAPM
O_RAPM = RAPM[RAPM['side'] == 'offense'][['ID','RAPM']]
O_RAPM = O_RAPM.rename(columns = {"RAPM":"O-RAPM"})
D_RAPM = RAPM[RAPM['side'] == 'defense'][['ID','RAPM']]
D_RAPM = D_RAPM.rename(columns = {"RAPM":"D-RAPM"})

components = O_RAPM.merge(right = D_RAPM, how = 'left', on = ['ID'])
names = RAPM[['ID','player_name']].drop_duplicates()
RAPM = names.merge(right = components, how = 'left', on = ['ID'])
RAPM['RAPM'] = RAPM['O-RAPM'] + RAPM['D-RAPM']
RAPM['SEASON'] = '2014-15'
RAPM.to_csv("C://Users/gsteele/Other/RAPM/2014_15_RAPM.csv")

In [401]:
print(len(unfiltered_RAPM[unfiltered_RAPM['side'] == 'defense']))
print(len(unfiltered_RAPM)/2)

492
490.5


In [414]:
#unfiltered_RAPM.sort_values(["RAPM"])
RAPM.sort_values('O-RAPM', ascending = False)

,ID,player_name,O-RAPM,D-RAPM,RAPM,SEASON
443,2422,John Salmons,1.172666,0.587570,1.760236,2014-15
30,101249,John Lucas III,1.071562,-0.793646,0.277916,2014-15
188,202355,Hassan Whiteside,0.983341,1.944669,2.928010,2014-15
466,2588,Matt Bonner,0.934283,3.079507,4.013790,2014-15
266,203105,Festus Ezeli,0.818589,0.928593,1.747182,2014-15
419,204037,Travis Wear,0.781928,0.998847,1.780775,2014-15
8,101122,Danny Granger,0.766075,1.525222,2.291297,2014-15
404,203958,Andre Dawkins,0.738614,-0.363187,0.375427,2014-15
45,200751,Randy Foye,0.708189,2.119595,2.827784,2014-15
479,2744,Al Jefferson,0.665092,1.353986,2.019078,2014-15


In [276]:
RAPM.sort_values('D-RAPM', ascending = False)

,ID,player_name,O-RAPM,D-RAPM,RAPM,SEASON
21,101150,Lou Williams,-0.394562,3.931724,3.537162,2014-15
38,1890,Shawn Marion,-1.216711,3.445522,2.228811,2014-15
320,203487,Michael Carter-Williams,1.452277,3.268573,4.720849,2014-15
274,203115,Will Barton,-1.203071,3.152588,1.949517,2014-15
417,204028,Tarik Black,-1.205306,3.094825,1.889519,2014-15
25,101181,Jose Calderon,0.456416,3.055301,3.511717,2014-15
455,2550,Kirk Hinrich,-3.399878,3.041195,-0.358683,2014-15
141,201956,Omri Casspi,-2.981074,3.012775,0.031701,2014-15
386,203932,Aaron Gordon,1.033840,2.963580,3.997421,2014-15
271,203112,Quincy Acy,0.022154,2.959920,2.982073,2014-15


In [262]:
RAPM.sort_values('RAPM', ascending = False)

,ID,player_name,O-RAPM,D-RAPM,RAPM,SEASON
9,101123,Gerald Green,2.923942,2.098371,5.022313,2014-15
86,201229,Anthony Tolliver,2.466459,2.504653,4.971112,2014-15
82,201188,Marc Gasol,1.709726,3.195726,4.905453,2014-15
203,202682,Derrick Williams,0.504164,4.374986,4.879150,2014-15
83,201196,Ramon Sessions,0.360333,4.095045,4.455378,2014-15
167,202324,Derrick Favors,2.689252,1.693478,4.382730,2014-15
307,203468,CJ McCollum,2.468260,1.808150,4.276410,2014-15
21,101150,Lou Williams,0.278745,3.993557,4.272301,2014-15
243,203078,Bradley Beal,2.263978,1.765344,4.029322,2014-15
323,203490,Otto Porter Jr.,0.151289,3.775989,3.927279,2014-15


In [165]:
RAPM.sort_values('RAPM', ascending = False)

,ID,player_name,O-RAPM,D-RAPM,RAPM,SEASON
132,201945,Gerald Henderson,2.497285,4.214787,6.712072,2014-15
107,201585,Kosta Koufos,1.101128,4.471184,5.572312,2014-15
151,201975,Jodie Meeks,1.672847,3.031394,4.704241,2014-15
155,201979,Nick Calathes,1.125444,3.559298,4.684743,2014-15
334,203504,Trey Burke,0.742988,3.664578,4.407567,2014-15
181,202338,Kevin Seraphin,-0.501410,4.636930,4.135520,2014-15
165,202322,John Wall,2.042612,2.048724,4.091336,2014-15
303,203460,Andre Roberson,0.642197,3.317398,3.959595,2014-15
441,2406,Caron Butler,1.506185,2.452511,3.958696,2014-15
376,203918,Rodney Hood,1.417677,2.513297,3.930974,2014-15


In [418]:
RAPM[RAPM['RAPM'].isna()]

,ID,player_name,O-RAPM,D-RAPM,RAPM,SEASON
233,202723,Malcolm Lee,NaN,NaN,NaN,2014-15
354,203564,Kalin Lucas,NaN,NaN,NaN,2014-15
423,204065,David Stockton,NaN,NaN,NaN,2014-15


1 (make), 2(miss), 3 (ft), 4 (reb), 5 (tov), 6(foul), 7(violation), 8(sub), 9(timeout), 10 (jumpball), 11 (ejection)

In [352]:
#stints.head()
#print(stints[stints['HOME_TEAM'] > stints['AWAY_TEAM']].count())
#print(stints[stints['AWAY_TEAM'] > stints['HOME_TEAM']].count())
#print(stints[stints['AWAY_TEAM'] == stints['HOME_TEAM']].count())
#len(stints[stints['stint'].str.contains('1628378')])
#len(stints[(stints['STINT_POSS'] == 0) & (stints['STINT_PTS'] > 0)])
#len(stints[stints['STINT_POSS'] == 0])
#stints[stints['STINT_POSS'] == 0].head(20)
#len(stints[(stints['STINT_POSS'] == 0) & (stints['STINT_PTS'] > 0)]['stint'].drop_duplicates())

#pbp_df[(pbp_df['stint'] == '201569, 201600, 201936, 201950, 203076, 202696, 202699, 203095, 203901, 203932')
#      & (pbp_df['game_id'] == '21400001') & (pbp_df['PERIOD'] == '2')]

#pbp[(pbp['GAME_ID'] == '21400001') & (pbp['PERIOD'] == '2')].sort_values('EVENTNUM')


#pbp_df[(pbp_df['stint'] == '101141, 101141, 201162, 201162, 202336, 202336, 202688, 202688, 203953, 203953, 101139, 101139, 201579, 201579, 202388, 202388, 203524, 203524, 2449, 2449')].sort_values('e')

#print(pbp_df[(pbp_df['homedescription'] != '') & (pbp_df['visitordescription'] != '')]['EVENTMSGTYPE'].value_counts())
#pbp_df[(pbp_df['homedescription'] != '') & (pbp_df['visitordescription'] != '') & (pbp_df['EVENTMSGTYPE'] == '2')].head()

#pbp_df[(pbp_df['EVENTMSGTYPE'] == '3') 
#                          & (pbp_df['DESCRIPTION'].str.contains('MISS'))
#                          & (pbp_df['NEXT_VISITORDESCRIPTION'].str.contains('REBOUND'))
#                          & (pbp_df['VISITORDESCRIPTION'] == '')
#                          & (pbp_df['POSS'] == 0)].head()